In [24]:
from src.luxai_s3.wrappers import LuxAIS3GymEnv
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math
from collections import deque
from torch.distributions import Categorical
from tqdm import tqdm

In [117]:
a = torch.zeros(5,2,6)
a[:,0] = 1
print(a)
a = torch.sum(a.flatten(0,1),axis=1)
print(a)
print(a.view(5,2))

tensor([[[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]]])
tensor([6., 0., 6., 0., 6., 0., 6., 0., 6., 0.])
tensor([[6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.]])


In [108]:
indices = torch.randperm(100).type(torch.IntTensor)
x = torch.rand((100))
x[indices[:100]]

tensor([0.9622, 0.5488, 0.1107, 0.0701, 0.7722, 0.5672, 0.3098, 0.7879, 0.5168,
        0.2152, 0.2722, 0.0164, 0.0052, 0.8149, 0.0077, 0.0638, 0.8164, 0.4512,
        0.0756, 0.6845, 0.3602, 0.6918, 0.2439, 0.1124, 0.7072, 0.1796, 0.3955,
        0.7336, 0.0934, 0.8547, 0.4365, 0.4686, 0.5946, 0.4098, 0.1832, 0.0687,
        0.1274, 0.3494, 0.0645, 0.9324, 0.2774, 0.6857, 0.1230, 0.1161, 0.7086,
        0.2544, 0.7184, 0.3671, 0.8880, 0.0355, 0.6036, 0.3076, 0.7279, 0.5499,
        0.0059, 0.0276, 0.7266, 0.2689, 0.1002, 0.4770, 0.9395, 0.6968, 0.2521,
        0.9680, 0.2555, 0.2303, 0.7571, 0.4251, 0.9588, 0.0950, 0.6509, 0.3164,
        0.7800, 0.7587, 0.1279, 0.5298, 0.8548, 0.6286, 0.9869, 0.9008, 0.0409,
        0.8541, 0.0707, 0.0888, 0.5810, 0.5028, 0.8040, 0.0835, 0.2401, 0.3443,
        0.7184, 0.0859, 0.8994, 0.6936, 0.7999, 0.7590, 0.3733, 0.1008, 0.2849,
        0.9459])

In [30]:
env = LuxAIS3GymEnv(numpy_output=True)
env.reset()

action = dict(
                player_0=np.random.randint(0,5,size=(env.env_params.max_units, 3)),
                player_1=np.random.randint(0,5,size=(env.env_params.max_units, 3))
            )
for i in range(10) :
    obs, reward, terminated, truncated, info = env.step(action)
print(reward)

{'player_0': Array(0, dtype=int32), 'player_1': Array(0, dtype=int32)}


In [132]:
def obs_to_state(obs,player) :

    n_units = len(obs['units']['position'][0])
    list_state = []
    
    if player == 'player_0' :
        #Units
        list_state.append(torch.from_numpy(obs['units']['position'].astype(float)).flatten()) #position
        list_state.append(torch.from_numpy(obs['units']['energy'].astype(float)).flatten()) #energy
        list_state.append(torch.from_numpy(obs['units_mask'].astype(float)).flatten()) #unit_mask
        
        #Map
        list_state.append(torch.from_numpy(obs['sensor_mask'].astype(float)).flatten()) #sensor_mask
        list_state.append(torch.from_numpy(obs['map_features']['energy'].astype(float)).flatten()) #map_energy
        list_state.append(torch.from_numpy(obs['map_features']['tile_type'].astype(float)).flatten()) #map_tile_type

        list_state.append(torch.from_numpy(obs['relic_nodes'].astype(float)).flatten()) #relic_nodes
        list_state.append(torch.from_numpy(obs['relic_nodes_mask'].astype(float)).flatten()) #relic_nodes_mask

        #Game
        print(torch.from_numpy(obs['team_points'].astype(float)))
        print(torch.from_numpy(obs['team_wins'].astype(float)))
        print(torch.from_numpy(obs['steps'].astype(float)))
        print(torch.from_numpy(obs['match_steps'].astype(float)))
        list_state.append(torch.from_numpy(obs['team_points'].astype(float)).flatten()) #team_points
        list_state.append(torch.from_numpy(obs['team_wins'].astype(float)).flatten()) #team_wins

        list_state.append(torch.from_numpy(obs['steps'].astype(float)).flatten()) #steps
        list_state.append(torch.from_numpy(obs['match_steps'].astype(float)).flatten()) #match_steps

    else :
        #Units
        list_state.append(torch.from_numpy(obs['units']['position'].astype(float))[[1,0]].flatten()) #position
        list_state.append(torch.from_numpy(obs['units']['energy'].astype(float))[[1,0]].flatten()) #energy
        list_state.append(torch.from_numpy(obs['units_mask'].astype(float))[[1,0]].flatten()) #unit_mask
        
        #Map
        list_state.append(torch.from_numpy(obs['sensor_mask'].astype(float)).flatten()) #sensor_mask
        list_state.append(torch.from_numpy(obs['map_features']['energy'].astype(float)).flatten()) #map_energy
        list_state.append(torch.from_numpy(obs['map_features']['tile_type'].astype(float)).flatten()) #map_tile_type

        list_state.append(torch.from_numpy(obs['relic_nodes'].astype(float)).flatten()) #relic_nodes
        list_state.append(torch.from_numpy(obs['relic_nodes_mask'].astype(float)).flatten()) #relic_nodes_mask

        #Game
        print(torch.from_numpy(obs['team_points'].astype(float)))
        print(torch.from_numpy(obs['team_wins'].astype(float)))
        print(torch.from_numpy(obs['steps'].astype(float)))
        print(torch.from_numpy(obs['match_steps'].astype(float)))
        list_state.append(torch.from_numpy(obs['team_points'].astype(float))[[1,0]].flatten()) #team_points
        list_state.append(torch.from_numpy(obs['team_wins'].astype(float))[[1,0]].flatten()) #team_wins

        list_state.append(torch.from_numpy(obs['steps'].astype(float)).flatten()) #steps
        list_state.append(torch.from_numpy(obs['match_steps'].astype(float)).flatten()) #match_steps
    
    return torch.cat(list_state)

state = obs_to_state(obs['player_0'],'player_0')
state = obs_to_state(obs['player_1'],'player_1')

tensor([ 0., 10.], dtype=torch.float64)
tensor([0., 0.], dtype=torch.float64)
tensor(50., dtype=torch.float64)
tensor(50., dtype=torch.float64)
tensor([ 0., 10.], dtype=torch.float64)
tensor([0., 0.], dtype=torch.float64)
tensor(50., dtype=torch.float64)
tensor(50., dtype=torch.float64)


In [121]:
class Policy(nn.Module) :
    def __init__(self,n_input,n_action,n_units,sap_range) :

        super(Policy,self).__init__()

        self.n_units = n_units
        self.n_action = n_action
        self.sap_range = sap_range

        self.inputs = nn.Linear(n_input,512,dtype=torch.double)

        self.hidden1 = nn.Linear(512,128,dtype=torch.double)
        self.hidden2 = nn.Linear(128,32,dtype=torch.double)

        self.actor_action = []
        self.actor_dx = []
        self.actor_dy = []

        self.critic_action = []
        self.critic_dx = []
        self.critic_dy = []

        for unit in range(self.n_units) :
            self.actor_action.append(nn.Linear(32,self.n_action,dtype=torch.double))
            self.actor_dx.append(nn.Linear(32,self.sap_range*2 + 1,dtype=torch.double))
            self.actor_dy.append(nn.Linear(32,self.sap_range*2 + 1,dtype=torch.double))

        self.critic = nn.Linear(32,1,dtype=torch.double)

    def forward(self,x) :

        x = F.relu(self.inputs(x))
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))

        actor_action = torch.zeros(self.n_units,self.n_action)
        actor_dx = torch.zeros(self.n_units,self.sap_range*2 + 1)
        actor_dy = torch.zeros(self.n_units,self.sap_range*2 + 1)

        for unit in range(self.n_units) :
            actor_action[unit] = F.log_softmax(self.actor_action[unit](x),dim=-1)
            actor_dx[unit] = F.log_softmax(self.actor_dx[unit](x),dim=-1)
            actor_dy[unit] = F.log_softmax(self.actor_dy[unit](x),dim=-1)

        value = self.critic(x)

        return actor_action,actor_dx,actor_dy,value

In [122]:
lr = 1e-5
n_input = 1880
n_units = env.env_params.max_units
sap_range = env.env_params.unit_sap_range
n_action = 6

model = Policy(n_input,n_action,n_units,sap_range=sap_range)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
num_episodes = 2
n = 1000
victory_bonus = 0

match_step = env.env_params.max_steps_in_match + 1
len_episode = match_step * env.env_params.match_count_per_episode


In [123]:
for i in range(n) :

    values = torch.zeros(2,num_episodes,len_episode)
    rewards = torch.zeros(2,num_episodes,len_episode)
    log_probs = torch.zeros(2,num_episodes,len_episode)

    for episode in range(1):
        # Reset the environment and get the initial state
        obs, _ = env.reset()
        state_0 = obs_to_state(obs['player_0'])
        state_1 = obs_to_state(obs['player_1'])

        base_reward_0 = 0
        base_reward_1 = 0
        # Keep track of the states, actions, and rewards for each step in the episode

        # Run the episode
        for step in range(50):

            # Get the action probabilities from the policy network
            actor_action_0,actor_dx_0,actor_dy_0, value_0 = model(state_0)
            actor_action_1,actor_dx_1,actor_dy_1, value_1 = model(state_1)

            action_0 = torch.zeros(n_units,3,dtype=torch.int)
            action_0[:,0] = Categorical(logits=actor_action_0).sample()
            action_0[:,1] = Categorical(logits=actor_dx_0).sample() - sap_range
            action_0[:,2] = Categorical(logits=actor_dy_0).sample() - sap_range

            action_1 = torch.zeros(n_units,3,dtype=torch.int)
            action_1[:,0] = Categorical(logits=actor_action_1).sample()
            action_1[:,1] = Categorical(logits=actor_dx_1).sample() - sap_range
            action_1[:,2] = Categorical(logits=actor_dy_1).sample() - sap_range

            log_prob_0 = torch.sum(actor_action_0[torch.arange(n_units),action_0[:,0]])
            log_prob_0 += torch.sum(actor_dx_0[torch.arange(n_units),action_0[:,1]])
            log_prob_0 += torch.sum(actor_dy_0[torch.arange(n_units),action_0[:,2]])

            log_prob_1 = torch.sum(actor_action_1[torch.arange(n_units),action_1[:,0]])
            log_prob_1 += torch.sum(actor_dx_1[torch.arange(n_units),action_1[:,1]])
            log_prob_1 += torch.sum(actor_dy_1[torch.arange(n_units),action_1[:,2]])

            action = dict(player_0 = np.array(action_0,dtype=int), player_1 = np.array(action_1,dtype=int))

            # Take the chosen action and observe the next state and reward
            obs, reward, truncated, done, info = env.step(action)
            next_state_0 = obs_to_state(obs['player_0'])
            next_state_1 = obs_to_state(obs['player_1'])

            if step == 0 :
                reward_memory = reward

            if reward['player_0'] > reward_memory['player_0'] or reward['player_1'] > reward_memory['player_1'] :
                base_reward_0 = reward_0
                base_reward_1 = reward_1

                if reward['player_0'] > reward_memory['player_0'] :
                    base_reward_0 += victory_bonus
                else : 
                    base_reward_1 += victory_bonus
                reward_memory = reward


            reward_0 = obs['player_0']['team_points'][0] + base_reward_0
            reward_1 = obs['player_1']['team_points'][1] + base_reward_1
                
            # Store the current state, action, and reward

            values[0,episode,step] = value_0
            values[1,episode,step] = value_1

            rewards[0,episode,step] = reward_0
            rewards[1,episode,step] = reward_1

            log_probs[0,episode,step] = log_prob_0
            log_probs[1,episode,step] = log_prob_1

            state_0 = next_state_0
            state_1 = next_state_1

    y = rewards[:,:,-1].view(2,num_episodes,1) - rewards

    y = y.flatten()
    values = values.flatten()
    log_probs = log_probs.flatten()

    policy_loss = torch.mean(log_probs * (y - values))
    value_loss = F.mse_loss(y,values)

    loss = policy_loss + value_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(base_reward_0,base_reward_1,loss)
        


torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16, 2])
torch.Size([2, 16])
torch.Size([

KeyboardInterrupt: 